In [1]:
import os
from PIL import Image
import numpy as np
import open3d as o3d
import cv2
from scipy.linalg import svd
import kornia
import torch
from unidepth.utils import colorize, image_grid
from unidepth.models import UniDepthV1
from unidepth.utils.visualization import save_file_ply, colorize_np

from os import listdir
from os.path import join
from unidepth.utils.geometric import project_points
from poselib import estimate_relative_pose
from kornia_moons.viz import draw_LAF_matches


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def apply_intrinsics(points, intrinsics):
    my_intrinsics = {'fx': intrinsics[0][0],
                    'fy': intrinsics[1][1],
                    'cx': intrinsics[0][2],
                    'cy': intrinsics[1][2], }
    x = points[:, 0]
    y = points[:, 1]
    z = points[:, 2]

    z = z  
    x = (x - my_intrinsics['cx']) * z / my_intrinsics['fx']
    y = (y - my_intrinsics['cy']) * z / my_intrinsics['fy']
    #print(x.shape)
    final = np.hstack((x.reshape(-1, 1), y.reshape(-1, 1), z.reshape(-1, 1)))
    #print(final.shape)
    return final

In [3]:
def estimate_initial_transform(source_points, target_points):
    Rt, scale = cv2.estimateAffine3D(source_points, target_points, force_rotation=True)
    print(scale)
    return Rt, scale

def compose_transform_matrix(Rt):
    row_to_add = np.array([0, 0, 0, 1])
    transform_matrix = np.vstack([Rt, row_to_add])
    return transform_matrix

In [4]:
def pairwise_pose(img1_path, img2_path, model):

    img1 = kornia.io.load_image(img1_path, kornia.io.ImageLoadType.RGB32)[None, ...]
    img2 = kornia.io.load_image(img2_path, kornia.io.ImageLoadType.RGB32)[None, ...]
    matcher = kornia.feature.LoFTR(pretrained="indoor")
    input_dict = {
        "image0": kornia.color.rgb_to_grayscale(img1),  # LofTR works on grayscale images only
        "image1": kornia.color.rgb_to_grayscale(img2),
    }
    
    with torch.inference_mode():
        correspondences = matcher(input_dict)

    mkpts1 = correspondences["keypoints0"].cpu().numpy()
    mkpts2 = correspondences["keypoints1"].cpu().numpy()

    #depth1 = np.load('im1depth.npy')
    #depth2 = np.load('im2depth.npy')

    rgb = np.array(Image.open(img1_path))
    rgb_torch = torch.from_numpy(rgb).permute(2, 0, 1)
    predictions = model.infer(rgb_torch)
    depth1 = predictions["depth"].squeeze().cpu().numpy()
    
    np.save('im1depth', depth1)
    H, W = rgb.shape[:2]
    points = predictions["points"].squeeze().cpu().permute(1,2,0).numpy().reshape(H*W, 3)
    new_rgb = rgb.reshape(H*W, 3)
    file = "ply2/im1.ply"
    save_file_ply(points, new_rgb, file)
    intrinsics1 = predictions["intrinsics"].squeeze().cpu().numpy()
    torch_intrinsic1 = predictions["intrinsics"]
    torch_pts1 = predictions["points"]

    rgb = np.array(Image.open(img2_path))
    rgb_torch = torch.from_numpy(rgb).permute(2, 0, 1)
    predictions = model.infer(rgb_torch)
    depth2 = predictions["depth"].squeeze().cpu().numpy()
    np.save('im2depth', depth2)
    H, W = rgb.shape[:2]
    points = predictions["points"].squeeze().cpu().permute(1,2,0).numpy().reshape(H*W, 3)
    new_rgb = rgb.reshape(H*W, 3)
    file = "ply2/im2.ply"
    save_file_ply(points, new_rgb, file)
    intrinsics2 = predictions["intrinsics"].squeeze().cpu().numpy()

    corresponding_points_3d_img1 = np.zeros((len(mkpts1), 3))
    for i, (x, y) in enumerate(mkpts1):
        depth = depth1[int(y), int(x)]
        corresponding_points_3d_img1[i] = [x, y, depth]
    
    corresponding_points_3d_img2 = np.zeros((len(mkpts2), 3))
    for i, (x, y) in enumerate(mkpts2):
        depth = depth2[int(y), int(x)]
        corresponding_points_3d_img2[i] = [x, y, depth]

    #print('Before intrinsics:', corresponding_points_3d_img1)
    #print('After intrinsics:', apply_intrinsics(corresponding_points_3d_img1, intrinsics1))
    
    cp1 = apply_intrinsics(corresponding_points_3d_img1, intrinsics1)
    cp2 = apply_intrinsics(corresponding_points_3d_img2, intrinsics2)

    Rt, scale = estimate_initial_transform(cp2, cp1)
   
  #  camera1 = {'model': 'SIMPLE_PINHOLE', 'width': 736, 'height': 468, 'params': [intrinsics1[0][0], intrinsics1[1][1],  intrinsics1[0][2],  intrinsics1[1][2]]}
  #  camera2 = {'model': 'SIMPLE_PINHOLE', 'width': 736, 'height': 468, 'params': [intrinsics2[0][0], intrinsics2[1][1],  intrinsics2[0][2],  intrinsics2[1][2]]}

  #  Rt2 = estimate_relative_pose(mkpts2, mkpts1, camera2, camera1)

    initial_transform_matrix = compose_transform_matrix(Rt)
    #print(initial_transform_matrix)


    return initial_transform_matrix, scale, torch_intrinsic1, torch_pts1

In [21]:
def create_depth_map(pointcloud, intrinsics_matrix, image_shape):
    # Project 3D points onto image plane
    projected_points, _ = cv2.projectPoints(pointcloud, np.eye(3), np.zeros(3), intrinsics_matrix, None)
    H, W = image_shape
    # Initialize depth map and count map
    depth_map = [[[] for _ in range(W)] for _ in range(H)]


    # Aggregate depth values that project to the same pixel
    for i, point in enumerate(projected_points):
        x, y = point.ravel()
        z = pointcloud[i, 2]

        if 0 <= x < image_shape[1] and 0 <= y < image_shape[0]:
            depth_map[int(y)][int(x)].append(z)
    
    final_depth = np.zeros(image_shape)

    m = 0.5
    for i in range(H):
        for j in range(W):
            data = np.array(depth_map[i][j])
            #if len(data) < 3:
            #    final_depth[i][j] = np.mean(data)
            #    continue
            #data = data[abs(data - np.mean(data)) < m * np.std(data)]
            median = np.median(data)
            final_depth[i][j] = median if median != np.NaN else 0

    return final_depth


In [ ]:
def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

In [12]:
def pairwise_registration(source, target, max_correspondence_distance_coarse, max_correspondence_distance_fine):
    print("Apply point-to-plane ICP")
    icp_coarse = o3d.pipelines.registration.registration_icp(
        source, target, max_correspondence_distance_coarse, np.identity(4),
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    icp_fine = o3d.pipelines.registration.registration_icp(
        source, target, max_correspondence_distance_fine,
        icp_coarse.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    transformation_icp = icp_fine.transformation
    information_icp = o3d.pipelines.registration.get_information_matrix_from_point_clouds(
        source, target, max_correspondence_distance_fine,
        icp_fine.transformation)
    return transformation_icp, information_icp


def full_registration(pcds, max_correspondence_distance_coarse,
                      max_correspondence_distance_fine):
    pose_graph = o3d.pipelines.registration.PoseGraph()
    odometry = np.identity(4)
    pose_graph.nodes.append(o3d.pipelines.registration.PoseGraphNode(odometry))
    n_pcds = len(pcds)
    for source_id in range(n_pcds):
        for target_id in range(source_id + 1, n_pcds):
            transformation_icp, information_icp = pairwise_registration(
                pcds[source_id], pcds[target_id], max_correspondence_distance_coarse, max_correspondence_distance_fine)
            print("Build o3d.pipelines.registration.PoseGraph")
            if target_id == source_id + 1:  # odometry case
                odometry = np.dot(transformation_icp, odometry)
                pose_graph.nodes.append(
                    o3d.pipelines.registration.PoseGraphNode(
                        np.linalg.inv(odometry)))
                pose_graph.edges.append(
                    o3d.pipelines.registration.PoseGraphEdge(source_id,
                                                             target_id,
                                                             transformation_icp,
                                                             information_icp,
                                                             uncertain=False))
            else:  # loop closure case
                pose_graph.edges.append(
                    o3d.pipelines.registration.PoseGraphEdge(source_id,
                                                             target_id,
                                                             transformation_icp,
                                                             information_icp,
                                                             uncertain=True))
    return pose_graph

In [6]:
from os import listdir
from os.path import join
import matplotlib.pyplot as plt
import copy

model = UniDepthV1.from_pretrained("lpiccinelli/unidepth-v1-vitl14")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

base_path = "base.png"
H,W  = np.array(Image.open(base_path)).shape[:2]

support_paths = []
for filename in listdir("supports"): 
    full_path = join("supports", filename)
    support_paths.append(full_path)


sup_pcds = []
for i, sup_path in enumerate(support_paths):
    T, scale, torch_intrinsics1, torch_points1 = pairwise_pose(base_path, sup_path, model) # base, sup order
    if scale > 1.1 or scale < 0.9:
        continue
    pcd1 = o3d.io.read_point_cloud("ply2/im1.ply")
    pcd2 = o3d.io.read_point_cloud("ply2/im2.ply")
    pcd2 = pcd2.scale(scale, center=(0, 0, 0))
    pcd2 = pcd2.transform(T)
    o3d.io.write_point_cloud(f"ply2/{i}.ply", pcd2)
    sup_pcds.append(pcd2)
   # os.remove("ply2/im1.ply")
   # os.remove("ply2/im2.ply")

Instantiate: dinov2_vitl14
0.9009178792806232
0.965194057866866
0.8596963478058745
0.9371253863070599
0.9117820714885012
0.9887373985294833
0.9226117237632034
0.846606878894724
0.9175268212750732
0.926433878167816
0.9539395769125829
0.9356813544818373
0.8907997893182353
0.9640348736512001
0.940428687178831
0.9672434315076213
0.9363565682915794
0.9740852097422532
1.0001081578176478
0.9006157704575346
0.9783318680773828
0.9406879540022874
0.8730625535236296
0.9687147521524532
0.9758516382683653
1.0226290750290676


In [14]:
voxel_size = 0.02
pcds_down = []
for pcd in sup_pcds:
    pcd_down = pcd.voxel_down_sample(voxel_size=voxel_size)
    radius_normal = voxel_size * 2
    pcd_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    pcds_down.append(pcd_down)

print("Full registration ...")
max_correspondence_distance_coarse = voxel_size * 15
max_correspondence_distance_fine = voxel_size * 1.5
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    pose_graph = full_registration(pcds_down,
                                   max_correspondence_distance_coarse,
                                   max_correspondence_distance_fine)

print("Optimizing PoseGraph ...")
option = o3d.pipelines.registration.GlobalOptimizationOption(
    max_correspondence_distance=max_correspondence_distance_fine,
    edge_prune_threshold=0.25,
    reference_node=0)
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    o3d.pipelines.registration.global_optimization(
        pose_graph,
        o3d.pipelines.registration.GlobalOptimizationLevenbergMarquardt(),
        o3d.pipelines.registration.GlobalOptimizationConvergenceCriteria(),
        option)

print("Transform points and display")
for point_id in range(len(pcds_down)):
    print(pose_graph.nodes[point_id].pose)
    pcds_down[point_id].transform(pose_graph.nodes[point_id].pose)

Full registration ...
Apply point-to-plane ICP
[Open3D DEBUG] ICP Iteration #0: Fitness 0.5990, RMSE 0.1423
[Open3D DEBUG] Residual : 1.46e-02 (# of elements : 132195)
[Open3D DEBUG] ICP Iteration #1: Fitness 0.6768, RMSE 0.1314
[Open3D DEBUG] Residual : 1.31e-02 (# of elements : 149374)
[Open3D DEBUG] ICP Iteration #2: Fitness 0.6760, RMSE 0.1046
[Open3D DEBUG] Residual : 7.09e-03 (# of elements : 149195)
[Open3D DEBUG] ICP Iteration #3: Fitness 0.6721, RMSE 0.0966
[Open3D DEBUG] Residual : 5.54e-03 (# of elements : 148334)
[Open3D DEBUG] ICP Iteration #4: Fitness 0.6696, RMSE 0.0944
[Open3D DEBUG] Residual : 5.17e-03 (# of elements : 147782)
[Open3D DEBUG] ICP Iteration #5: Fitness 0.6680, RMSE 0.0935
[Open3D DEBUG] Residual : 5.03e-03 (# of elements : 147427)
[Open3D DEBUG] ICP Iteration #6: Fitness 0.6667, RMSE 0.0930
[Open3D DEBUG] Residual : 4.96e-03 (# of elements : 147154)
[Open3D DEBUG] ICP Iteration #7: Fitness 0.6659, RMSE 0.0928
[Open3D DEBUG] Residual : 4.94e-03 (# of elem

In [7]:
pcd_comb = copy.deepcopy(pcd1)
for pcd in sup_pcds:
    pcd_comb += pcd

In [29]:
depth_comb = create_depth_map(np.asarray(pcd_comb.points), torch_intrinsics1.squeeze().cpu().numpy(), (H,W))
depth_comb = cv2.medianBlur(depth_comb.astype('float32'), 5)

depth_col_avg = colorize_np(depth_comb, vmin=0.01, vmax=10.0, cmap="magma_r")
Image.fromarray(depth_col_avg).save("averaged4.png")

points_tensor = torch.tensor(np.asarray(pcd_comb.points), dtype=torch.float)
points_tensor = points_tensor.view(-1, 3).unsqueeze(0).cuda()
torch_depth = project_points(points_tensor, torch_intrinsics1, (H,W))
depth_pred_col = colorize(torch_depth.squeeze().cpu().numpy(), vmin=0.01, vmax=10.0, cmap="magma_r")
Image.fromarray(depth_pred_col).save("averaged.png")

point_cloud_flat = torch_points1.view(1, 3, -1)  # Shape: (B, 3, HW)
point_cloud_transposed = point_cloud_flat.transpose(1, 2)  # Shape: (B, HW, 3)
torch_depth = project_points(point_cloud_transposed, torch_intrinsics1, (H,W))
depth_pred_col = colorize(torch_depth.squeeze().cpu().numpy(), vmin=0.01, vmax=10.0, cmap="magma_r")
Image.fromarray(depth_pred_col).save("original.png") 

In [74]:
model = UniDepthV1.from_pretrained("lpiccinelli/unidepth-v1-vitl14")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

img1 = kornia.io.load_image('pair2/im0.png', kornia.io.ImageLoadType.RGB32)[None, ...]
img2 = kornia.io.load_image('pair2/im1.png', kornia.io.ImageLoadType.RGB32)[None, ...]
    
_ , _ , H,W = img1.size()

T, torch_intrinsics1, torch_points1 = pairwise_pose(img1,img2, model)
pcd1 = o3d.io.read_point_cloud("ply2/im1.ply")
pcd2 = o3d.io.read_point_cloud("ply2/im2.ply")
pcd2 = pcd2.transform(T)


""" voxel_size = 0.1

source_down, source_fpfh = preprocess_point_cloud(pcd2, voxel_size)
target_down, target_fpfh = preprocess_point_cloud(pcd1, voxel_size)

result_ransac = execute_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh,
                                            voxel_size)

pcd2 = pcd2.transform(result_ransac.transformation)  """

o3d.io.write_point_cloud("ply2/im2new.ply", pcd2)

pcd_comb = pcd1 + pcd2

points = np.asarray(pcd_comb.points)
points_tensor = torch.tensor(points, dtype=torch.float)
points_tensor = points_tensor.view(-1, 3).unsqueeze(0).cuda()
depth = project_points(points_tensor, torch_intrinsics1, (H,W))
depth_pred_col = colorize(depth.squeeze().cpu().numpy(), vmin=0.01, vmax=10.0, cmap="magma_r")
Image.fromarray(depth_pred_col).save("averaged.png")

point_cloud_flat = torch_points1.view(1, 3, -1)  # Shape: (B, 3, HW)
point_cloud_transposed = point_cloud_flat.transpose(1, 2)  # Shape: (B, HW, 3)
depth = project_points(point_cloud_transposed, torch_intrinsics1, (H,W))
depth_pred_col = colorize(depth.squeeze().cpu().numpy(), vmin=0.01, vmax=10.0, cmap="magma_r")
Image.fromarray(depth_pred_col).save("original.png")



""" points = np.asarray(pcd_comb.points)
points_tensor = torch.tensor(points, dtype=torch.float)
points_tensor = points_tensor.view(-1, 3).unsqueeze(0).cuda()
torch_depth = project_points(points_tensor, torch_intrinsics1, (H,W))
depth_pred_col = colorize(torch_depth.squeeze().cpu().numpy(), vmin=0.01, vmax=10.0, cmap="magma_r")
Image.fromarray(depth_pred_col).save("averaged.png")

point_cloud_flat = torch_points1.view(1, 3, -1)  # Shape: (B, 3, HW)
point_cloud_transposed = point_cloud_flat.transpose(1, 2)  # Shape: (B, HW, 3)
torch_depth = project_points(point_cloud_transposed, torch_intrinsics1, (H,W))
depth_pred_col = colorize(torch_depth.squeeze().cpu().numpy(), vmin=0.01, vmax=10.0, cmap="magma_r")
Image.fromarray(depth_pred_col).save("original.png") """


""" depth = create_depth_map(np.asarray(pcd1.points), intrinsics1.squeeze().cpu().numpy(), (H,W))
depth_comb = create_depth_map(np.asarray(pcd_comb.points), intrinsics1.squeeze().cpu().numpy(), (H,W))

plt.imshow(depth, cmap='jet')
plt.colorbar()
plt.title('Depth Map')
plt.savefig('depth.png') 
plt.clf()

plt.imshow(depth_comb, cmap='jet')
plt.colorbar()
plt.title('Depth Map')
plt.savefig('depth_avg.png') 
plt.clf() """

""" points = np.asarray(pcd_comb.points)
points_tensor = torch.tensor(points, dtype=torch.float)
points_tensor = points_tensor.view(-1, 3).unsqueeze(0).cuda()

#print(points_tensor)
depth = project_points(points_tensor, intrinsics1, (H,W))
print(depth.shape)
depth_pred_col = colorize(depth.squeeze().cpu().numpy(), vmin=0.01, vmax=10.0, cmap="magma_r")
Image.fromarray(depth_pred_col).save("averaged.png")
 """

Instantiate: dinov2_vitl14


TypeError: expected str, bytes or os.PathLike object, not Tensor